In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/NLP/final_project/final_project/fp-dataset-artifacts-main
# /NLP/final_project/fp-dataset-artifacts-main/

/content/gdrive/MyDrive/NLP/final_project/final_project/fp-dataset-artifacts-main


In [ ]:
pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
predicted_dt = pd.read_json(path_or_buf='/content/gdrive/MyDrive/NLP/final_project/final_project/fp-dataset-artifacts-main/eval_output/eval_predictions.jsonl', lines=True)

In [ ]:
len(predicted_dt)

9842

In [ ]:
import json
prediction_path = '/content/gdrive/MyDrive/NLP/final_project/final_project/fp-dataset-artifacts-main/eval_output/eval_predictions.jsonl'

# Open and read the JSONL file
with open(prediction_path, 'r') as f:
    json_list = []
    for line in f:
        json_obj = json.loads(line)  # Parse each line as a JSON object
        json_list.append(json_obj)

In [ ]:
n = len(json_list)
for i in range(n):
  if json_list[i]['label'] != json_list[i]['predicted_label']:
    json_list[i]['result'] = 'wrong'
  else:
    json_list[i]['result'] = 'right'

In [ ]:
json_list[0]

{'premise': 'Two women are embracing while holding to go packages.',
 'hypothesis': 'The sisters are hugging goodbye while holding to go packages after just eating lunch.',
 'label': 1,
 'predicted_scores': [-2.53448748588562, 4.505809307098389, -2.77573561668396],
 'predicted_label': 1,
 'result': 'right'}

In [ ]:
from collections import defaultdict
import re
from itertools import islice

# Tokenizer function to split text into tokens
def simple_tokenizer(text):
    # Tokenize by splitting on spaces and removing punctuation
    tokens = re.findall(r'\b\w+\b', text.lower())
    return tokens

# Function to extract n-grams from tokenized text
def get_ngrams(tokens, n):
    return zip(*(islice(tokens, i, None) for i in range(n)))

# Function to count n-grams in premise, hypothesis by label
def count_ngrams_by_label(json_file, n=2):
    # Dictionary to store n-gram counts by label (entailment, contradiction, neutral)
    ngram_counts_by_label = defaultdict(lambda: defaultdict(int))

    # Read the JSON file
    # with open(json_file, 'r') as f:
    #     data = json.load(f)

    # Loop through each example in the SNLI data
    for example in json_file:
        premise = example['premise']
        hypothesis = example['hypothesis']
        label = example['predicted_label']

        # Tokenize the premise and hypothesis
        premise_tokens = simple_tokenizer(premise)
        hypothesis_tokens = simple_tokenizer(hypothesis)

        # Optionally, combine premise and hypothesis tokens for analysis
        combined_tokens = premise_tokens + hypothesis_tokens

        # Extract n-grams from combined tokens
        ngrams = get_ngrams(combined_tokens, n)

        # Count the n-grams for this label
        for ngram in ngrams:
            ngram_str = ' '.join(ngram)  # Join n-gram tokens into a string
            ngram_counts_by_label[label][ngram_str] += 1

    return ngram_counts_by_label

def count_ngrams_by_result(json_file, n=2):
    # Dictionary to store n-gram counts by label (entailment, contradiction, neutral)
    ngram_counts_by_label = defaultdict(lambda: defaultdict(int))

    # Read the JSON file
    # with open(json_file, 'r') as f:
    #     data = json.load(f)

    # Loop through each example in the SNLI data
    for example in json_file:
        premise = example['premise']
        hypothesis = example['hypothesis']
        label = example['result']

        # Tokenize the premise and hypothesis
        premise_tokens = simple_tokenizer(premise)
        hypothesis_tokens = simple_tokenizer(hypothesis)

        # Optionally, combine premise and hypothesis tokens for analysis
        combined_tokens = premise_tokens + hypothesis_tokens

        # Extract n-grams from combined tokens
        ngrams = get_ngrams(combined_tokens, n)

        # Count the n-grams for this label
        for ngram in ngrams:
            ngram_str = ' '.join(ngram)  # Join n-gram tokens into a string
            ngram_counts_by_label[label][ngram_str] += 1

    return ngram_counts_by_label

In [ ]:
ngram_counts_1 = count_ngrams_by_label(json_list, n=1)
ngram_counts_1_result = count_ngrams_by_result(json_list, n=1)

In [ ]:
ngram_counts_1.keys(), ngram_counts_1_result.keys()

(dict_keys([1, 0, 2]), dict_keys(['right', 'wrong']))

In [ ]:
pd.DataFrame(ngram_counts_1.items())

,0,1
0,1,"{'two': 713, 'women': 211, 'are': 1207, 'embra..."
1,0,"{'two': 805, 'women': 233, 'are': 1307, 'embra..."
2,2,"{'two': 729, 'women': 236, 'are': 1173, 'embra..."


In [ ]:
pd.DataFrame(ngram_counts_1_result.items())

,0,1
0,right,"{'two': 2000, 'women': 602, 'are': 3295, 'embr..."
1,wrong,"{'two': 247, 'young': 134, 'children': 54, 'in..."


In [ ]:
ngram_counts_1_0 = pd.DataFrame(ngram_counts_1.get(0).items()).rename(columns={0:'token',1:'0'})
ngram_counts_1_1 = pd.DataFrame(ngram_counts_1.get(1).items()).rename(columns={0:'token',1:'1'})
ngram_counts_1_2 = pd.DataFrame(ngram_counts_1.get(2).items()).rename(columns={0:'token',1:'2'})

data1 = {key: [value] for key, value in ngram_counts_1_result.get('wrong').items()}
ngram_counts_1_result_w = pd.DataFrame(data1).T.reset_index().rename(columns={'index':'token',0:'wrong'})
data2 = {key: [value] for key, value in ngram_counts_1_result.get('right').items()}
ngram_counts_1_result_r = pd.DataFrame(data2).T.reset_index().rename(columns={'index':'token',0:'right'})

In [ ]:
len(ngram_counts_1_0), len(ngram_counts_1_1), len(ngram_counts_1_2), len(ngram_counts_1_result_w), len(ngram_counts_1_result_r)

(4266, 5044, 4854, 2874, 6001)

In [ ]:
ngram_counts_1_dt = ngram_counts_1_0.merge(ngram_counts_1_1, how='outer').merge(ngram_counts_1_2, how='outer').fillna(0)

In [ ]:
ngram_counts_1_result_dt = ngram_counts_1_result_w.merge(ngram_counts_1_result_r, how='outer').fillna(0)

In [ ]:
ngram_counts_1_dt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3529 entries, 0 to 4265
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   token   3529 non-null   object 
 1   0       3529 non-null   float64
 2   1       3529 non-null   float64
 3   2       3529 non-null   float64
dtypes: float64(3), object(1)
memory usage: 137.9+ KB


In [ ]:
ngram_counts_1_result_dt

,token,wrong,right
0,two,247.0,2000.0
1,young,134.0,1071.0
2,children,54.0,444.0
3,in,821.0,7122.0
4,blue,98.0,937.0
...,...,...,...
6290,markings,0.0,3.0
6291,aerial,0.0,6.0
6292,barista,0.0,1.0
6293,zebra,0.0,3.0


In [ ]:
ngram_counts_1_dt['sum'] = ngram_counts_1_dt['0'] + ngram_counts_1_dt['1'] + ngram_counts_1_dt['2']
ngram_counts_1_dt['diff1'] = abs(ngram_counts_1_dt['0']-ngram_counts_1_dt['1'])
ngram_counts_1_dt['diff2'] = abs(ngram_counts_1_dt['0']-ngram_counts_1_dt['2'])
ngram_counts_1_dt['diff3'] = abs(ngram_counts_1_dt['2']-ngram_counts_1_dt['1'])
ngram_counts_1_dt['diff_max'] = ngram_counts_1_dt[['diff1','diff2','diff3']].max(axis=1)
ngram_counts_1_dt['perc'] = ngram_counts_1_dt['diff_max'] / ngram_counts_1_dt['sum']

In [ ]:
ngram_counts_1_dt_sort = ngram_counts_1_dt.sort_values('perc', ascending=False)

In [ ]:
pd.options.display.max_rows=6000

In [ ]:
ngram_counts_1_comp = ngram_counts_1_dt_sort.merge(ngram_counts_1_result_dt, how='left')

In [ ]:
ngram_counts_1_comp[ngram_counts_1_comp['token'].isin(['shine','hammers','hate','fists'])][['token','0','1','2','wrong','right']].rename(columns={'0':'#predicted label 0','1':'#predicted label 1', '2':'#predicted label 2', 'wrong':'#predicted wrong','right':'#predicted right'})

,token,#predicted label 0,#predicted label 1,#predicted label 2,#predicted wrong,#predicted right
650,shine,2.0,0.0,0.0,2.0,0.0
1897,hammers,0.0,0.0,4.0,3.0,1.0
2615,hate,1.0,2.0,5.0,4.0,4.0
2797,fists,0.0,1.0,1.0,2.0,0.0


In [ ]:
len(ngram_counts_1_comp)

NameError: name 'ngram_counts_1_comp' is not defined

In [ ]:
ngram_counts_1_comp[(ngram_counts_1_comp['perc']>=0.5)&(ngram_counts_1_comp['wrong']>=ngram_counts_1_comp['right'])&(ngram_counts_1_comp['sum']>1)]

,token,0,1,2,sum,diff1,diff2,diff3,diff_max,perc,wrong,right
17,connie,0.0,3.0,0.0,3.0,3.0,0.0,3.0,3.0,1.000000,2.0,1.0
18,facial,0.0,4.0,0.0,4.0,4.0,0.0,4.0,4.0,1.000000,2.0,2.0
19,tissues,0.0,3.0,0.0,3.0,3.0,0.0,3.0,3.0,1.000000,2.0,1.0
20,sandcastle,0.0,2.0,0.0,2.0,2.0,0.0,2.0,2.0,1.000000,1.0,1.0
27,sword,0.0,7.0,0.0,7.0,7.0,0.0,7.0,7.0,1.000000,4.0,3.0
75,ben,0.0,2.0,0.0,2.0,2.0,0.0,2.0,2.0,1.000000,1.0,1.0
103,policewoman,0.0,2.0,0.0,2.0,2.0,0.0,2.0,2.0,1.000000,2.0,0.0
104,st,0.0,2.0,0.0,2.0,2.0,0.0,2.0,2.0,1.000000,1.0,1.0
105,patrick,0.0,2.0,0.0,2.0,2.0,0.0,2.0,2.0,1.000000,1.0,1.0
106,ina,0.0,2.0,0.0,2.0,2.0,0.0,2.0,2.0,1.000000,1.0,1.0


In [ ]:
ngram_counts_2 = count_ngrams_by_label(json_list, n=2)
ngram_counts_2_result = count_ngrams_by_result(json_list, n=2)

In [ ]:
ngram_counts_2_0 = pd.DataFrame(ngram_counts_2.get(0).items()).rename(columns={0:'token',1:'0'})
ngram_counts_2_1 = pd.DataFrame(ngram_counts_2.get(1).items()).rename(columns={0:'token',1:'1'})
ngram_counts_2_2 = pd.DataFrame(ngram_counts_2.get(2).items()).rename(columns={0:'token',1:'2'})

data1 = {key: [value] for key, value in ngram_counts_2_result.get('wrong').items()}
ngram_counts_2_result_w = pd.DataFrame(data1).T.reset_index().rename(columns={'index':'token',0:'wrong'})
data2 = {key: [value] for key, value in ngram_counts_2_result.get('right').items()}
ngram_counts_2_result_r = pd.DataFrame(data2).T.reset_index().rename(columns={'index':'token',0:'right'})

In [ ]:
len(ngram_counts_2_0), len(ngram_counts_2_1), len(ngram_counts_2_2), len(ngram_counts_2_result_w), len(ngram_counts_2_result_r)

(21940, 25156, 23758, 10405, 33470)

In [ ]:
ngram_counts_2_dt = ngram_counts_2_0.merge(ngram_counts_2_1, how='outer').merge(ngram_counts_2_2, how='outer').fillna(0)
ngram_counts_2_result_dt = ngram_counts_2_result_w.merge(ngram_counts_2_result_r, how='outer').fillna(0)

In [ ]:
ngram_counts_2_dt['sum'] = ngram_counts_2_dt['0'] + ngram_counts_2_dt['1'] + ngram_counts_2_dt['2']
ngram_counts_2_dt['diff1'] = abs(ngram_counts_2_dt['0']-ngram_counts_2_dt['1'])
ngram_counts_2_dt['diff2'] = abs(ngram_counts_2_dt['0']-ngram_counts_2_dt['2'])
ngram_counts_2_dt['diff3'] = abs(ngram_counts_2_dt['2']-ngram_counts_2_dt['1'])
ngram_counts_2_dt['diff_max'] = ngram_counts_2_dt[['diff1','diff2','diff3']].max(axis=1)
ngram_counts_2_dt['perc'] = ngram_counts_2_dt['diff_max'] / ngram_counts_2_dt['sum']
ngram_counts_2_dt_sort = ngram_counts_2_dt.sort_values('perc', ascending=False)

In [ ]:
ngram_counts_2_comp = ngram_counts_2_dt_sort.merge(ngram_counts_2_result_dt, how='left')

In [ ]:
ngram_counts_2_comp[ngram_counts_2_comp['token']=='wait for']

,token,0,1,2,sum,diff1,diff2,diff3,diff_max,perc,wrong,right
14857,wait for,0.0,10.0,2.0,12.0,10.0,2.0,8.0,10.0,0.833333,2.0,10.0


In [ ]:
ngram_counts_2_comp[(ngram_counts_2_comp['perc']>0.5)&(ngram_counts_2_comp['wrong']>=ngram_counts_2_comp['right'])&(ngram_counts_2_comp['sum']>1)]

,token,0,1,2,sum,diff1,diff2,diff3,diff_max,perc,wrong,right
25,the square,0.0,2.0,0.0,2.0,2.0,0.0,2.0,2.0,1.000000,1.0,1.0
35,pictures the,0.0,2.0,0.0,2.0,2.0,0.0,2.0,2.0,1.000000,1.0,1.0
65,wooden bars,0.0,2.0,0.0,2.0,2.0,0.0,2.0,2.0,1.000000,1.0,1.0
330,are sword,0.0,3.0,0.0,3.0,3.0,0.0,3.0,3.0,1.000000,2.0,1.0
331,sword fighting,0.0,3.0,0.0,3.0,3.0,0.0,3.0,3.0,1.000000,2.0,1.0
332,fighting two,0.0,2.0,0.0,2.0,2.0,0.0,2.0,2.0,1.000000,2.0,0.0
335,sword fight,0.0,3.0,0.0,3.0,3.0,0.0,3.0,3.0,1.000000,2.0,1.0
508,to media,0.0,3.0,0.0,3.0,3.0,0.0,3.0,3.0,1.000000,2.0,1.0
509,media during,0.0,3.0,0.0,3.0,3.0,0.0,3.0,3.0,1.000000,2.0,1.0
510,crowded protest,0.0,3.0,0.0,3.0,3.0,0.0,3.0,3.0,1.000000,2.0,1.0


In [ ]:
from scipy.stats import chi2_contingency
import numpy as np
e = 0.00001
def run_chi_square_test(ngram_label_count):
    chi_square_results = []
    n = len(ngram_label_count)
    for i in range(n):
        # Create contingency table as a list of counts [entailment, neutral, contradiction]
        contingency_table = [ngram_label_count.loc[i,'0']+e, ngram_label_count.loc[i,'1']+e]#, ngram_label_count.loc[i,'2']+e]
        # Run chi-square test on the contingency table
        chi2, p_value, _, _ = chi2_contingency([contingency_table])
        chi_square_results.append((ngram_label_count.loc[i,'token'], chi2, p_value))
    return chi_square_results

In [ ]:
ngram_1_chi_square = run_chi_square_test(ngram_counts_1_comp)
ngram_2_chi_square = run_chi_square_test(ngram_counts_2_comp)

In [ ]:
for i in range(len(ngram_1_chi_square)):
  if ngram_1_chi_square[i][2]!=1:
    print(ngram_1_chi_square[i])

In [ ]:
predicted_wrong = predicted_dt[predicted_dt['predicted_label']!=predicted_dt['label']].reset_index()

In [ ]:
predicted_wrong.to_csv('predicted_wrong.csv')

In [ ]:
predicted_right = predicted_dt[predicted_dt['predicted_label']==predicted_dt['label']].reset_index()

In [ ]:
1-len(predicted_wrong)/len(predicted_dt),len(predicted_right)/len(predicted_dt)

(0.8942288152814468, 0.8942288152814468)

In [ ]:
predicted_wrong.loc[0,'premise'],predicted_wrong.loc[0,'hypothesis'], predicted_wrong.loc[0,'label'],predicted_wrong.loc[0,'predicted_label']

('Two young children in blue jerseys, one with the number 9 and one with the number 2 are standing on wooden steps in a bathroom and washing their hands in a sink.',
 'Two kids at a ballgame wash their hands.',
 1,
 2)

In [ ]:
predicted_wrong.loc[0,'predicted_scores']

[-3.196985721588134, 0.765240788459777, 2.074043035507202]

In [ ]:
predicted_wrong[['label','predicted_label']].groupby('label').value_counts()

label  predicted_label
0      1                  251
       2                   59
1      0                  221
       2                  213
2      1                  224
       0                   73
Name: count, dtype: int64

In [ ]:
predicted_right[['label','predicted_label']].groupby('label').value_counts()

,,count
label,predicted_label,
0,0,3019
1,1,2801
2,2,2981


In [ ]:
predicted_dt.head()

,premise,hypothesis,label,predicted_scores,predicted_label
0,Two women are embracing while holding to go pa...,The sisters are hugging goodbye while holding ...,1,"[-2.53448748588562, 4.505809307098389, -2.7757...",1
1,Two women are embracing while holding to go pa...,Two woman are holding packages.,0,"[3.903407096862793, -1.59943425655365, -3.4539...",0
2,Two women are embracing while holding to go pa...,The men are fighting outside a deli.,2,"[-4.000194549560547, -2.508118152618408, 6.207...",2
3,"Two young children in blue jerseys, one with t...",Two kids in numbered jerseys wash their hands.,0,"[3.693380832672119, -1.809481739997863, -2.930...",0
4,"Two young children in blue jerseys, one with t...",Two kids at a ballgame wash their hands.,1,"[-3.196985721588134, 0.765240788459777, 2.0740...",2


In [ ]:
import nltk
from collections import defaultdict
from nltk.util import ngrams
nltk.download('punkt')  # Download the Punkt tokenizer models
import string
# def get_ngrams(text, n):
#     tokens = text.split()
#     return list(ngrams(tokens, n))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def tokenize(text):
    # Tokenize text using nltk's word_tokenize
    return nltk.word_tokenize(text)

def get_ngrams(text, n):
    # Generate n-grams from tokenized words
    tokens = tokenize(text)
    return list(ngrams([word.lower() for word in tokens if word not in string.punctuation], n)) #[word.lower() for word in tokens if word not in string.punctuation]

# Suppose dataset is a list of (premise, hypothesis, label) tuples
# def count_ngrams_in_dataset(dataset, n):
#     ngram_label_count = defaultdict(lambda: {'entailment': 0, 'neutral': 0, 'contradiction': 0})
#     # ngram_label_count = defaultdict(lambda: {'0': 0, '1': 0, '2': 0})
#     dt_n = len(dataset)
#     for i in range(dt_n):
#         premise_ngrams = get_ngrams(dataset.loc[i,'premise'], n)
#         hypothesis_ngrams = get_ngrams(dataset.loc[i,'hypothesis'], n)
#         label = dataset.loc[0,'predicted_label']
#         label = 'entailment' if label == 0 else 'neutral' if label == 1 else 'contradiction'
#         # print(i,premise_ngrams,hypothesis_ngrams,label)
#         for ngram in premise_ngrams + hypothesis_ngrams:
#             # print(ngram)
#             ngram_label_count[ngram][label] += 1
#             # ngram_label_count[ngram[0], label] += 1
#     return ngram_label_count

def count_ngrams_in_dataset(dataset, n):
    """
    This function counts the frequency of n-grams for each label.

    :param dataset: List of (premise, hypothesis, label) tuples.
    :param n: The 'n' in n-grams (e.g., 1 for unigrams, 2 for bigrams, etc.).
    :return: Dictionary with n-gram counts across labels.
    """
    # Initialize frequency table to count occurrences of each n-gram in each label
    ngram_label_count = defaultdict(lambda: {'entailment': 0, 'neutral': 0, 'contradiction': 0})

    for row in dataset:
        # Get n-grams for both premise and hypothesis
        premise_ngrams = get_ngrams(row[0], n)
        hypothesis_ngrams = get_ngrams(row[1], n)

        # Combine the n-grams from premise and hypothesis
        all_ngrams = premise_ngrams + hypothesis_ngrams

        # Update the counts for each n-gram based on the label
        for ngram in all_ngrams:
            ngram_label_count[ngram][row[2]] += 1

    return ngram_label_count

In [ ]:
predicted_dt.iloc[:,[0,1,4]]

,premise,hypothesis,predicted_label
0,Two women are embracing while holding to go pa...,The sisters are hugging goodbye while holding ...,neutral
1,Two women are embracing while holding to go pa...,Two woman are holding packages.,entailment
2,Two women are embracing while holding to go pa...,The men are fighting outside a deli.,contradiction
3,"Two young children in blue jerseys, one with t...",Two kids in numbered jerseys wash their hands.,entailment
4,"Two young children in blue jerseys, one with t...",Two kids at a ballgame wash their hands.,contradiction
...,...,...,...
9837,A small girl wearing a pink jacket is riding o...,The carousel is moving.,entailment
9838,A small girl wearing a pink jacket is riding o...,The girl is moving at the speed of light.,entailment
9839,A young girl with blue and pink ribbons in her...,People in a water fountain,entailment
9840,A young girl with blue and pink ribbons in her...,A young girl knits a sweater,contradiction


In [ ]:
predicted_dt.loc[predicted_dt['predicted_label'] == 0,'predicted_label'] = 'entailment'
predicted_dt.loc[predicted_dt['predicted_label'] == 1,'predicted_label'] = 'neutral'
predicted_dt.loc[predicted_dt['predicted_label'] == 2,'predicted_label'] = 'contradiction'

<ipython-input-14-168791658c2f>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'entailment' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  predicted_dt.loc[predicted_dt['predicted_label'] == 0,'predicted_label'] = 'entailment'


In [ ]:
predicted_dt_list = list(zip(*[predicted_dt[col] for col in predicted_dt.iloc[:,[0,1,4]]]))

In [ ]:
get_ngrams(predicted_dt.loc[0,'premise'], 1)

[('two',),
 ('women',),
 ('are',),
 ('embracing',),
 ('while',),
 ('holding',),
 ('to',),
 ('go',),
 ('packages',)]

In [ ]:
get_ngrams(predicted_dt.loc[0,'premise'], 2)

[('two', 'women'),
 ('women', 'are'),
 ('are', 'embracing'),
 ('embracing', 'while'),
 ('while', 'holding'),
 ('holding', 'to'),
 ('to', 'go'),
 ('go', 'packages')]

In [ ]:
predicted_dt_label_count_1_dt = pd.DataFrame(predicted_dt_label_count_1).T

In [ ]:
from scipy.stats import fisher_exact

def run_fisher_test(ngram_label_count):
    fisher_results = []
    n = len(ngram_label_count)
    for i in range(n):
        # Create contingency table as a list of counts [entailment, neutral, contradiction]
        contingency_table = [ngram_label_count.loc[i,'0'], ngram_label_count.loc[i,'1'], ngram_label_count.loc[i,'2']]
        # Run chi-square test on the contingency table
        oddsratio, p_val = fisher_exact(contingency_table)
        fisher_results.append((ngram_label_count.loc[i,'token'], oddsratio, p_val))
    return fisher_results

# Example 2x2 table with a zero count
# contingency_table = np.array([[10, 0],  # N-gram present in entailment
#                               [100, 80]])  # N-gram absent

# # Fisher's Exact Test for 2x2 tables
# oddsratio, p_val = fisher_exact(contingency_table)

In [ ]:
chi_square_results = run_chi_square_test(predicted_dt_label_count_1)

In [ ]:
chi_square_results = sorted(chi_square_results, key=lambda x: x[1], reverse=True)

In [ ]:
chi_square_results[-1]

('zeropos', 0.0, 1.0)

In [ ]:
predicted_dt_label_count_1_dt['total_count'] = predicted_dt_label_count_1_dt['contradiction'] + predicted_dt_label_count_1_dt['neutral'] + predicted_dt_label_count_1_dt['entailment']

In [ ]:
predicted_dt_label_count_1_dt[(predicted_dt_label_count_1_dt['contradiction']!=0)&(predicted_dt_label_count_1_dt['neutral']!=0)]

,entailment,neutral,contradiction,total_count


In [ ]:
predicted_dt_label_count_1

defaultdict(<function __main__.count_ngrams_in_dataset.<locals>.<lambda>()>,
            {('two',): {'entailment': 0, 'neutral': 2238, 'contradiction': 0},
             ('women',): {'entailment': 0, 'neutral': 679, 'contradiction': 0},
             ('are',): {'entailment': 0, 'neutral': 3688, 'contradiction': 0},
             ('embracing',): {'entailment': 0,
              'neutral': 6,
              'contradiction': 0},
             ('while',): {'entailment': 0,
              'neutral': 1282,
              'contradiction': 0},
             ('holding',): {'entailment': 0,
              'neutral': 657,
              'contradiction': 0},
             ('to',): {'entailment': 0, 'neutral': 2231, 'contradiction': 0},
             ('go',): {'entailment': 0, 'neutral': 48, 'contradiction': 0},
             ('packages',): {'entailment': 0,
              'neutral': 7,
              'contradiction': 0},
             ('the',): {'entailment': 0, 'neutral': 9762, 'contradiction': 0},
             (

In [ ]:
predicted_dt

,premise,hypothesis,label,predicted_scores,predicted_label
0,Two women are embracing while holding to go pa...,The sisters are hugging goodbye while holding ...,1,"[-2.53448748588562, 4.505809307098389, -2.7757...",1
1,Two women are embracing while holding to go pa...,Two woman are holding packages.,0,"[3.903407096862793, -1.59943425655365, -3.4539...",0
2,Two women are embracing while holding to go pa...,The men are fighting outside a deli.,2,"[-4.000194549560547, -2.508118152618408, 6.207...",2
3,"Two young children in blue jerseys, one with t...",Two kids in numbered jerseys wash their hands.,0,"[3.693380832672119, -1.809481739997863, -2.930...",0
4,"Two young children in blue jerseys, one with t...",Two kids at a ballgame wash their hands.,1,"[-3.196985721588134, 0.765240788459777, 2.0740...",2
...,...,...,...,...,...
9837,A small girl wearing a pink jacket is riding o...,The carousel is moving.,0,"[3.016649961471557, -0.225844353437423, -4.161...",0
9838,A small girl wearing a pink jacket is riding o...,The girl is moving at the speed of light.,2,"[2.818933248519897, -0.24153429269790602, -3.8...",0
9839,A young girl with blue and pink ribbons in her...,People in a water fountain,0,"[4.130252361297607, -1.333585023880004, -4.053...",0
9840,A young girl with blue and pink ribbons in her...,A young girl knits a sweater,2,"[-3.920511960983276, -1.884308815002441, 5.339...",2


In [ ]:
!python run.py --do_train --task nli --dataset snli --output_dir ./trained_model/

2024-09-14 18:01:17.762100: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-14 18:01:17.790346: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-14 18:01:17.798559: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-14 18:01:17.824130: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-14 18:01:19.332180: W tensorflow/compiler/tf2

In [ ]:
# !python run.py --do_eval --task nli --dataset snli --model ./trained_model/ --output_dir ./eval_output/
!python run.py --do_eval --task nli --dataset snli --model ./electra_hans_finetuned/ --output_dir ./hans_eval_output/

2024-10-28 20:34:11.140901: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-28 20:34:11.162413: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-28 20:34:11.168571: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 20:34:12.439608: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/gdrive/MyDrive/NLP/final_project/final_project/fp-dataset-artifacts-main/run.py", line 209, in <module>
    main()
  File "/content/gdrive/MyDrive/NLP/final_project/final_project/fp-dataset

In [ ]:
import numpy as np
import collections
from collections import defaultdict, OrderedDict
from transformers import Trainer, EvalPrediction
from transformers.trainer_utils import PredictionOutput
from typing import Tuple
from tqdm.auto import tqdm

QA_MAX_ANSWER_LENGTH = 30


# This function preprocesses an NLI dataset, tokenizing premises and hypotheses.
def prepare_dataset_nli(examples, tokenizer, max_seq_length=None):
    max_seq_length = tokenizer.model_max_length if max_seq_length is None else max_seq_length

    tokenized_examples = tokenizer(
        examples['premise'],
        examples['hypothesis'],
        truncation=True,
        max_length=max_seq_length,
        padding='max_length'
    )

    tokenized_examples['label'] = examples['label']
    return tokenized_examples


# This function computes sentence-classification accuracy.
# Functions with signatures like this one work as the "compute_metrics" argument of transformers.Trainer.
def compute_accuracy(eval_preds: EvalPrediction):
    return {
        'accuracy': (np.argmax(
            eval_preds.predictions,
            axis=1) == eval_preds.label_ids).astype(
            np.float32).mean().item()
    }


# This function preprocesses a question answering dataset, tokenizing the question and context text
# and finding the right offsets for the answer spans in the tokenized context (to use as labels).
# Adapted from https://github.com/huggingface/transformers/blob/master/examples/pytorch/question-answering/run_qa.py
def prepare_train_dataset_qa(examples, tokenizer, max_seq_length=None):
    questions = [q.lstrip() for q in examples["question"]]
    max_seq_length = tokenizer.model_max_length
    # tokenize both questions and the corresponding context
    # if the context length is longer than max_length, we split it to several
    # chunks of max_length
    tokenized_examples = tokenizer(
        questions,
        examples["context"],
        truncation="only_second",
        max_length=max_seq_length,
        stride=min(max_seq_length // 2, 128),
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    # Since one example might give us several features if it has a long context,
    # we need a map from a feature to its corresponding example.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position
    # in the original context. This will help us compute the start_positions
    # and end_positions to get the final answer string.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        # We will label features not containing the answer the index of the CLS token.
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)
        # from the feature idx to sample idx
        sample_index = sample_mapping[i]
        # get the answer for a feature
        answers = examples["answers"][sample_index]

        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and
                    offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and \
                        offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(
                    token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


def prepare_validation_dataset_qa(examples, tokenizer):
    questions = [q.lstrip() for q in examples["question"]]
    max_seq_length = tokenizer.model_max_length
    tokenized_examples = tokenizer(
        questions,
        examples["context"],
        truncation="only_second",
        max_length=max_seq_length,
        stride=min(max_seq_length // 2, 128),
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
    # corresponding example_id and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples


# This function uses start and end position scores predicted by a question answering model to
# select and extract the predicted answer span from the context.
# Adapted from https://github.com/huggingface/transformers/blob/master/examples/pytorch/question-answering/utils_qa.py
def postprocess_qa_predictions(examples,
                               features,
                               predictions: Tuple[np.ndarray, np.ndarray],
                               n_best_size: int = 20):
    if len(predictions) != 2:
        raise ValueError(
            "`predictions` should be a tuple with two elements (start_logits, end_logits).")
    all_start_logits, all_end_logits = predictions

    if len(predictions[0]) != len(features):
        raise ValueError(
            f"Got {len(predictions[0])} predictions and {len(features)} features.")

    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[
            example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    all_predictions = collections.OrderedDict()

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        prelim_predictions = []

        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits
            # to span of texts in the original context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[
                            -1: -n_best_size - 1: -1].tolist()
            end_indexes = np.argsort(end_logits)[
                          -1: -n_best_size - 1: -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                            start_index >= len(offset_mapping)
                            or end_index >= len(offset_mapping)
                            or offset_mapping[start_index] is None
                            or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or \
                            end_index - start_index + 1 > QA_MAX_ANSWER_LENGTH:
                        continue

                    prelim_predictions.append(
                        {
                            "offsets": (offset_mapping[start_index][0],
                                        offset_mapping[end_index][1]),
                            "score": start_logits[start_index] +
                                     end_logits[end_index],
                            "start_logit": start_logits[start_index],
                            "end_logit": end_logits[end_index],
                        }
                    )

        # Only keep the best `n_best_size` predictions.
        predictions = sorted(prelim_predictions, key=lambda x: x["score"],
                             reverse=True)[:n_best_size]

        # Use the offsets to gather the answer text in the original context.
        context = example["context"]
        for pred in predictions:
            offsets = pred.pop("offsets")
            pred["text"] = context[offsets[0]: offsets[1]]

        # In the very rare edge case we have not a single non-null prediction,
        # we create a fake prediction to avoid failure.
        if len(predictions) == 0 or (
                len(predictions) == 1 and predictions[0]["text"] == ""):
            predictions.insert(0, {"text": "empty", "start_logit": 0.0,
                                   "end_logit": 0.0, "score": 0.0})

        all_predictions[example["id"]] = predictions[0]["text"]
    return all_predictions


# Adapted from https://github.com/huggingface/transformers/blob/master/examples/pytorch/question-answering/trainer_qa.py
class QuestionAnsweringTrainer(Trainer):
    def __init__(self, *args, eval_examples=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.eval_examples = eval_examples

    def evaluate(self,
                 eval_dataset=None,  # denotes the dataset after mapping
                 eval_examples=None,  # denotes the raw dataset
                 ignore_keys=None,  # keys to be ignored in dataset
                 metric_key_prefix: str = "eval"
                 ):
        eval_dataset = self.eval_dataset if eval_dataset is None else eval_dataset
        eval_dataloader = self.get_eval_dataloader(eval_dataset)
        eval_examples = self.eval_examples if eval_examples is None else eval_examples

        # Temporarily disable metric computation, we will do it in the loop here.
        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        try:
            # compute the raw predictions (start_logits and end_logits)
            output = self.evaluation_loop(
                eval_dataloader,
                description="Evaluation",
                # No point gathering the predictions if there are no metrics, otherwise we defer to
                # self.args.prediction_loss_only
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
            )
        finally:
            self.compute_metrics = compute_metrics

        if self.compute_metrics is not None:
            # post process the raw predictions to get the final prediction
            # (from start_logits, end_logits to an answer string)
            eval_preds = postprocess_qa_predictions(eval_examples,
                                                    eval_dataset,
                                                    output.predictions)
            formatted_predictions = [{"id": k, "prediction_text": v}
                                     for k, v in eval_preds.items()]
            references = [{"id": ex["id"], "answers": ex['answers']}
                          for ex in eval_examples]

            # compute the metrics according to the predictions and references
            metrics = self.compute_metrics(
                EvalPrediction(predictions=formatted_predictions,
                               label_ids=references)
            )

            # Prefix all keys with metric_key_prefix + '_'
            for key in list(metrics.keys()):
                if not key.startswith(f"{metric_key_prefix}_"):
                    metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)

            self.log(metrics)
        else:
            metrics = {}

        self.control = self.callback_handler.on_evaluate(self.args, self.state,
                                                         self.control, metrics)
        return metrics

In [ ]:
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
    AutoModelForQuestionAnswering, Trainer, TrainingArguments, HfArgumentParser
from helpers import prepare_dataset_nli, prepare_train_dataset_qa, \
    prepare_validation_dataset_qa, QuestionAnsweringTrainer, compute_accuracy
import os
import json

import torch

NUM_PREPROCESSING_WORKERS = 2

In [ ]:
dataset = 'snli'
task = 'nli'
default_datasets = {'qa': ('squad',), 'nli': ('snli',)}
dataset_id = tuple(dataset.split(':')) if dataset is not None else \
            default_datasets[task]
# MNLI has two validation splits (one with matched domains and one with mismatched domains). Most datasets just have one "validation" split
eval_split = 'validation_matched' if dataset_id == ('glue', 'mnli') else 'validation'
# Load the raw data
dataset = datasets.load_dataset(*dataset_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/412k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/413k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/19.6M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/550152 [00:00<?, ? examples/s]

In [ ]:
commonsense_qa = datasets.load_dataset("commonsense_qa")

README.md:   0%|          | 0.00/7.39k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/160k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9741 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1221 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1140 [00:00<?, ? examples/s]

In [ ]:
challenge_set = datasets.load_dataset("hans")

README.md:   0%|          | 0.00/7.02k [00:00<?, ?B/s]

hans.py:   0%|          | 0.00/5.22k [00:00<?, ?B/s]

The repository for hans contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hans.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("./trained_model", num_labels=3)

In [ ]:
def preprocess_hans(example):
    encoding = tokenizer(example['premise'], example['hypothesis'], truncation=True, padding="max_length", max_length=128)
    encoding['labels'] = 0 if example['label'] == "entailment" else 2  # 0 for entailment, 2 for non-entailment
    return encoding

# Process HANS dataset
hans = challenge_set.map(preprocess_hans)
hans.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Training arguments for HANS
training_args_hans = TrainingArguments(
    output_dir="./electra_hans",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=2,
)

# Initialize Trainer for HANS
trainer_hans = Trainer(
    model=model,
    args=training_args_hans,
    train_dataset=hans['train'],
    eval_dataset=hans['validation'],
    compute_metrics=compute_accuracy,
)

# Train on HANS
trainer_hans.train()
trainer_hans.save_model()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.000000,0.000000,1.000000
2,0.000000,0.000000,1.000000


In [ ]:
challenge_set['train'][7]

{'premise': 'The senator believed the secretary .',
 'hypothesis': 'The secretary believed the senator .',
 'label': 1,
 'parse_premise': '(ROOT (S (NP (DT The) (NN senator)) (VP (VBD believed) (NP (DT the) (NN secretary))) (. .)))',
 'parse_hypothesis': '(ROOT (S (NP (DT The) (NN secretary)) (VP (VBD believed) (NP (DT the) (NN senator))) (. .)))',
 'binary_parse_premise': '( ( The senator ) ( ( believed ( the secretary ) ) . ) )',
 'binary_parse_hypothesis': '( ( The secretary ) ( ( believed ( the senator ) ) . ) )',
 'heuristic': 'lexical_overlap',
 'subcase': 'ln_subject/object_swap',
 'template': 'temp1'}

In [ ]:
dataset['train']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [ ]:
task_kwargs = {'num_labels': 3}
model_class = AutoModelForSequenceClassification
model = model_class.from_pretrained('./trained_model/', **task_kwargs)

In [ ]:
for param in model.parameters(): param.data = param.data.contiguous() # add this forthe contiguous error
tokenizer = AutoTokenizer.from_pretrained('./trained_model/', use_fast=True)

In [ ]:
prepare_train_dataset = prepare_eval_dataset = \
            lambda exs: prepare_dataset_nli(exs, tokenizer, 128)

In [ ]:
dataset = dataset.filter(lambda ex: ex['label'] != -1)

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/550152 [00:00<?, ? examples/s]

In [ ]:
eval_dataset = None
eval_dataset_featurized = None
eval_dataset = dataset[eval_split] #adversarial
eval_dataset_featurized = eval_dataset.map(
    prepare_eval_dataset,
    batched=True,
    num_proc=NUM_PREPROCESSING_WORKERS,
    remove_columns=eval_dataset.column_names
)

Map (num_proc=2):   0%|          | 0/9842 [00:00<?, ? examples/s]

In [ ]:
def predict_label(premise, hypothesis):
    # Encode the premise and hypothesis
    inputs = tokenizer.encode_plus(premise, hypothesis, return_tensors='pt')
    # Get the model's output
    outputs = model(**inputs)
    logits = outputs.logits
    # The prediction will be the class with the highest logit score
    predicted_class = torch.argmax(logits, dim=1).item()
    return predicted_class

In [ ]:
# adversarial_1 = {'premise': 'A cat is chasing a mouse.',
#  'hypothesis': 'A cat is being chased by a mouse.',
#  'label': 2}
# adversarial_2 = {'premise': 'A man is standing next to a car.',
#  'hypothesis': 'The man is neither standing nor sitting.',
#  'label': 2}
# adversarial_3 = {'premise': 'A baby is sleeping in a crib.',
#  'hypothesis': 'A baby is dreaming.',
#  'label': 2}
# adversarial_4 = {'premise': 'The man is holding a saxophone',
#  'hypothesis': 'he man is holding an electric guitar',
#  'label': 2}
# adversarial_5 = {'premise': 'A little girl is very sad.',
#  'hypothesis': 'A little girl is very unhappy.',
#  'label': 0}
# adversarial_6 = {'premise': 'The lawyer was advised by the actor.',
#  'hypothesis': ' The actor advised the lawyer.',
#  'label': 0}
# adversarial_7 = {'premise': 'Before the actor slept, the senator ran.',
#  'hypothesis': 'The actor slept.',
#  'label': 0}
adversarial_1 = {'premise':'A man is playing the piano.', 'hypothesis':	'A person is not performing music.', 'label':	'Contradiction'}
adversarial_2 = {'premise':'A woman is cooking dinner.','hypothesis':	'A woman is not preparing food.', 'label':	'Contradiction'}
adversarial_3 = {'premise':'Two children are playing in the park.', 'hypothesis':	'The children are studying inside a classroom.', 'label':	'Contradiction'}
adversarial_4 = {'premise':'A cat is chasing a mouse.', 'hypothesis':	'A cat is being chased by a mouse.', 'label':	'Contradiction'}
adversarial_5 = {'premise':'A man is standing next to a car.', 'hypothesis':	'The man is neither standing nor sitting.', 'label':	'Neutral or Contradiction'}
adversarial_6 = {'premise':'A baby is sleeping in a crib.', 'hypothesis':	'A baby is dreaming.', 'label':	'Neutral'}

In [ ]:
(predict_label(adversarial_1['premise'],adversarial_1['hypothesis']),
predict_label(adversarial_2['premise'],adversarial_2['hypothesis']),
predict_label(adversarial_3['premise'],adversarial_3['hypothesis']),
 predict_label(adversarial_4['premise'],adversarial_4['hypothesis']),
predict_label(adversarial_5['premise'],adversarial_5['hypothesis']),
predict_label(adversarial_6['premise'],adversarial_6['hypothesis']))

(2, 2, 2, 0, 0, 1)

In [ ]:
(predict_label(adversarial_1['premise'],adversarial_1['hypothesis']),
predict_label(adversarial_2['premise'],adversarial_2['hypothesis']),
predict_label(adversarial_3['premise'],adversarial_3['hypothesis']))

(0, 0, 1)

In [ ]:
predict_label(adversarial_7['premise'],adversarial_7['hypothesis'])

0

In [ ]:
dataset['test'].features

{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None)}

In [ ]:
dataset['test'].features['label']

ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None)

In [ ]:
dataset['test'][0]

{'premise': 'This church choir sings to the masses as they sing joyous songs from the book at a church.',
 'hypothesis': 'The church has cracks in the ceiling.',
 'label': 1}

In [ ]:
dataset['test']['label'][:3]

[1, 0, 2]

In [ ]:
test_labels = dataset['test']['label']
label_names = dataset['test'].features['label'].int2str(test_labels[:3])
label_names

['neutral', 'entailment', 'contradiction']

In [ ]:
dataset['train'][0]

{'premise': 'A person on a horse jumps over a broken down airplane.',
 'hypothesis': 'A person is training his horse for a competition.',
 'label': 1}

In [ ]:
task_kwargs = {'num_labels': 3}
max_length = 128
model = './trained_model/'
tokenizer = AutoTokenizer.from_pretrained(model, use_fast=True)
prepare_train_dataset = prepare_eval_dataset = \
            lambda exs: prepare_dataset_nli(exs, tokenizer, max_length)

In [ ]:
if dataset_id == ('snli',):
    # remove SNLI examples with no label
    dataset = dataset.filter(lambda ex: ex['label'] != -1)

train_dataset = None
eval_dataset = None
train_dataset_featurized = None
eval_dataset_featurized = None
# if training_args.do_train:
train_dataset = dataset['train']
# if args.max_train_samples:
#     train_dataset = train_dataset.select(range(args.max_train_samples))
train_dataset_featurized = train_dataset.map(
    prepare_train_dataset,
    batched=True,
    num_proc=NUM_PREPROCESSING_WORKERS,
    remove_columns=train_dataset.column_names
)
# if training_args.do_eval:
eval_dataset = dataset[eval_split]
# if args.max_eval_samples:
#     eval_dataset = eval_dataset.select(range(args.max_eval_samples))
eval_dataset_featurized = eval_dataset.map(
    prepare_eval_dataset,
    batched=True,
    num_proc=NUM_PREPROCESSING_WORKERS,
    remove_columns=eval_dataset.column_names
)

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/550152 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/549367 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/9842 [00:00<?, ? examples/s]

In [ ]:
train_dataset_featurized

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 549367
})

In [ ]:
eval_dataset_featurized

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 9842
})